# TitanBlock Phase II: Target Definition (Final)

## Step 1: Structural Setup & Biological Dimer Extraction
**Final Logic:** 
1. **Isolate Ligand:** Extract Chain A (GDF11).
2. **Symmetry Scan:** Generate mates within 25 Å.
3. **Triple-Factor Selection:** 
   - **Maximize Contacts:** Must share a large interface.
   - **Geometric Sanity:** COM distance 10-30 Å.
   - **Clash Guard:** Reject mates with > 5 steric clashes (atoms < 1.2 Å apart). This prevents physically impossible overlaps.

In [5]:
import os
import urllib.request
import pymol
from pymol import cmd

# 1. Download PDBs
pdbs = {"MSTN": "3HH2", "GDF11": "6MAC"}
for name, code in pdbs.items():
    url = f"https://files.rcsb.org/download/{code}.pdb"
    filename = f"{code}.pdb"
    if not os.path.exists(filename):
        print(f"Downloading {name}...")
        urllib.request.urlretrieve(url, filename)

# Initialize PyMOL
pymol.finish_launching(['pymol', '-cq'])
cmd.reinitialize()

# --- PROCESSING MYOSTATIN (TARGET) ---
print("Loading Myostatin...")
cmd.load("3HH2.pdb", "mstn_raw")
cmd.create("mstn_clean", "mstn_raw and chain A+B and not solvent")
print("MSTN cleaned.")

# --- PROCESSING GDF11 ---
print("Processing GDF11...")
cmd.load("6MAC.pdb", "gdf11_full")

# 1. Extract ONLY Chain A (Ligand)
cmd.create("gdf11_monomer", "gdf11_full and chain A and not solvent")
cmd.delete("gdf11_full") 

# 2. Generate ALL symmetry mates within 25 Angstroms
cmd.symexp("sym", "gdf11_monomer", "gdf11_monomer", 25.0)

# 3. ROBUST MATE SELECTION (Contacts + Clashes)
mates = [x for x in cmd.get_object_list("sym*") if x != "gdf11_monomer"]

best_mate = None
max_contacts = 0
best_stats = ""

com_anchor = cmd.centerofmass("gdf11_monomer")

print(f"Scanning {len(mates)} symmetry mates...")

for m in mates:
    # Metric A: Contacts (Primary Signal)
    contacts = cmd.count_atoms(f"({m}) within 4.0 of (gdf11_monomer)")
    
    # Metric B: Clashes (Safety Guard)
    clashes = cmd.count_atoms(f"({m}) within 1.2 of (gdf11_monomer)")
    
    # Metric C: Geometry (COM Distance)
    com_mate = cmd.centerofmass(m)
    dist = ((com_anchor[0]-com_mate[0])**2 + 
            (com_anchor[1]-com_mate[1])**2 + 
            (com_anchor[2]-com_mate[2])**2)**0.5
    
    print(f"Mate {m}: {contacts} contacts, {clashes} clashes, Dist {dist:.1f} Å")
    
    # LOGIC: 10-30A dist, minimal clashes (<5), maximize contacts
    if 10.0 < dist < 30.0 and clashes < 5:
        if contacts > max_contacts:
            max_contacts = contacts
            best_mate = m
            best_stats = f"Contacts: {contacts}, Clashes: {clashes}, Dist: {dist:.1f} Å"

if best_mate and max_contacts > 20:
    print(f"SUCCESS: Biological Partner Identified: {best_mate} ({best_stats})")
    cmd.alter(best_mate, "chain='B'")
    cmd.create("gdf11_clean", f"gdf11_monomer or {best_mate}")
else:
    print("CRITICAL ERROR: No valid biological dimer found.")

# --- ANCHOR ALIGNMENT ---
if "mstn_clean" in cmd.get_object_list("all") and "gdf11_clean" in cmd.get_object_list("all"):
    # Align Anchor A to Anchor A
    rms = cmd.super("gdf11_clean and chain A", "mstn_clean and chain A", cycles=5)
    print(f"PyMOL Anchor RMSD: {rms[0]:.3f} Angstroms")
    
    cmd.save("mstn_clean.pdb", "mstn_clean")
    cmd.save("gdf11_aligned.pdb", "gdf11_clean")
    print("Aligned files saved.")
else:
    print("Error: Structures not ready.")

cmd.delete("all")

Loading Myostatin...
MSTN cleaned.
Processing GDF11...
 Symmetry: Found 6 symmetry operators.
Scanning 10 symmetry mates...
Mate sym020000-1: 0 contacts, 0 clashes, Dist 58.6 Å
Mate sym050000-1: 0 contacts, 0 clashes, Dist 59.2 Å
Mate sym01000000: 1 contacts, 0 clashes, Dist 48.7 Å
Mate sym02000000: 2 contacts, 0 clashes, Dist 48.7 Å
Mate sym03000000: 0 contacts, 0 clashes, Dist 36.4 Å
Mate sym04000000: 66 contacts, 0 clashes, Dist 19.0 Å
Mate sym05000000: 0 contacts, 0 clashes, Dist 56.4 Å
Mate sym01000001: 0 contacts, 0 clashes, Dist 58.6 Å
Mate sym04000001: 0 contacts, 0 clashes, Dist 46.6 Å
Mate sym050100-1: 0 contacts, 0 clashes, Dist 73.3 Å
SUCCESS: Biological Partner Identified: sym04000000 (Contacts: 66, Clashes: 0, Dist: 19.0 Å)
PyMOL Anchor RMSD: 0.681 Angstroms
Aligned files saved.


## Step 2: Identify Interface (Target Side)
**Refined Logic:** We calculate SASA specifically for **Chain B (Target)** in the context of the dimer. This ensures our exposure metrics exactly match the residues we are ranking.

In [6]:
import os
from Bio.PDB import PDBParser, NeighborSearch, is_aa
from Bio.PDB.SASA import ShrakeRupley

# Note: We scan Chain B (Target) against Chain A (Anchor)
def identify_interface(pdb_file, output_file="interface_residues.txt", chain1="B", chain2="A"):
    if not os.path.exists(pdb_file): return
    
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("TARGET", pdb_file)
    model = structure[0]
    
    sr = ShrakeRupley()
    try: sr.compute(model, level="R")
    except: pass

    # Find Interface
    atoms2 = [a for a in model[chain2].get_atoms() if a.element != "H"]
    ns = NeighborSearch(atoms2)
    
    exposed_interface = []
    
    print(f"{'Residue':<10} | {'SASA':<6} | {'Status'}")
    print("-" * 30)
    
    for r in model[chain1]:
        if not is_aa(r, standard=True): continue
        
        is_interface = False
        for a in r.get_atoms():
            if len(ns.search(a.get_coord(), 5.0)) > 0:
                is_interface = True
                break
        
        if is_interface:
            sasa = getattr(r, "sasa", 0.0)
            if sasa > 10.0:
                exposed_interface.append((r.get_resname(), r.id[1], sasa))
                print(f"{r.get_resname()} {r.id[1]:<4} | {sasa:.1f}   | KEEP")
                
    with open(output_file, "w") as f:
        f.write("ResName,SeqID,SASA\n")
        for res, seq, sasa in exposed_interface:
            f.write(f"{res},{seq},{sasa}\n")
    
    print("-" * 30)
    print(f"Saved {len(exposed_interface)} exposed interface residues (Chain {chain1}).")

identify_interface("mstn_clean.pdb")

Residue    | SASA   | Status
------------------------------
LEU 20   | 26.3   | KEEP
PHE 27   | 40.1   | KEEP
TRP 29   | 56.5   | KEEP
TYR 38   | 20.5   | KEEP
ASN 41   | 20.5   | KEEP
TYR 42   | 16.3   | KEEP
CYS 43   | 22.9   | KEEP
SER 44   | 14.2   | KEEP
GLY 45   | 35.9   | KEEP
TYR 55   | 78.0   | KEEP
PRO 56   | 41.9   | KEEP
HIS 59   | 43.6   | KEEP
VAL 61   | 19.3   | KEEP
GLN 63   | 102.3   | KEEP
ALA 64   | 37.9   | KEEP
PRO 66   | 24.5   | KEEP
ARG 67   | 206.1   | KEEP
GLY 68   | 53.6   | KEEP
SER 69   | 30.4   | KEEP
CYS 73   | 20.7   | KEEP
CYS 74   | 25.2   | KEEP
PRO 76   | 33.0   | KEEP
MET 79   | 42.3   | KEEP
ILE 98   | 45.9   | KEEP
PRO 99   | 86.7   | KEEP
ALA 100  | 12.9   | KEEP
VAL 102  | 21.7   | KEEP
SER 109  | 20.2   | KEEP
------------------------------
Saved 28 exposed interface residues (Chain B).


## Step 3: Specificity Ranking (Geometry + Chemistry)
**New Logic:** 
1. **Chemistry Gate:** We identify residues that swap properties (e.g. Charge Flip) between Myostatin and GDF11.
2. **Selection Rule:** A residue is a hotspot if it has a large shift (>1.5 Å) **OR** a high-impact chemical change.
3. **Pruning:** We keep only the Top 12 candidates to focus the generative model.

In [7]:
import Bio.PDB
from Bio.PDB import is_aa
import numpy as np
import os

TOP_K = 12

def calculate_displacement(atom1, atom2):
    diff = atom1.get_coord() - atom2.get_coord()
    return np.sqrt(np.sum(diff * diff))

def aa_class(res_name):
    pos = {"LYS","ARG","HIS"}
    neg = {"ASP","GLU"}
    polar = {"SER","THR","ASN","GLN","TYR","CYS"}
    hyd = {"ALA","VAL","ILE","LEU","MET","PHE","TRP","PRO","GLY"}
    if res_name in pos: return "pos"
    if res_name in neg: return "neg"
    if res_name in polar: return "polar"
    if res_name in hyd: return "hyd"
    return "other"

def analyze_shifts(mstn_pdb, gdf11_pdb, interface_file):
    sasa_map = {}
    if os.path.exists(interface_file):
        with open(interface_file, "r") as f:
            next(f)
            for line in f:
                if line.strip():
                    parts = line.split(',')
                    sasa_map[int(parts[1])] = float(parts[2])

    parser = Bio.PDB.PDBParser(QUIET=True)
    mstn = parser.get_structure("MSTN", mstn_pdb)[0]
    gdf11 = parser.get_structure("GDF11", gdf11_pdb)[0]
    
    # 1. Verification Check
    sq_diff_sum = 0
    count = 0
    for r in mstn['A']:
        if is_aa(r) and 'CA' in r:
            rid = r.id[1]
            if rid in gdf11['A'] and 'CA' in gdf11['A'][rid]:
                d = calculate_displacement(r['CA'], gdf11['A'][rid]['CA'])
                sq_diff_sum += d*d
                count += 1
    
    rmsd = np.sqrt(sq_diff_sum / count) if count > 0 else 99.9
    print(f"Verification RMSD: {rmsd:.3f} Å over {count} CA pairs")
    
    # 2. Analyze Target (Chain B)
    print(f"\n{'Res':<5} | {'M-G':<7} | {'Type':<5} | {'Shift':<7} | {'SASA':<5} | {'Score':<5} | {'Verdict'}")
    print("-" * 70)
    
    candidates = []
    
    for resid, sasa_val in sasa_map.items():
        if resid in mstn['B'] and resid in gdf11['B']:
            try:
                atom_m = mstn['B'][resid]['CA']
                atom_g = gdf11['B'][resid]['CA']
                res_m = mstn['B'][resid].get_resname()
                res_g = gdf11['B'][resid].get_resname()
                
                dist = calculate_displacement(atom_m, atom_g)
                
                # Chemistry Check
                type_m = aa_class(res_m)
                type_g = aa_class(res_g)
                chem_change = (type_m != type_g)
                
                # Selection Logic (OR Gate)
                is_geom = (1.5 < dist < 8.0)
                is_chem = (chem_change and dist < 8.0)
                
                # Scoring: Boost chemical changes to compete with geometric shifts
                eff_dist = dist
                if is_chem and dist < 3.0: eff_dist = 3.0 # Bonus for specificity
                score = eff_dist * sasa_val
                
                if is_geom or is_chem:
                    tag = "GEOM" if is_geom else "CHEM"
                    if is_geom and is_chem: tag = "BOTH"
                    
                    candidates.append({
                        'id': resid, 'shift': dist, 'score': score,
                        'res_str': f"{res_m}-{res_g}", 'tag': tag, 'sasa': sasa_val
                    })
                    
            except KeyError: pass

    # Sort and Prune
    candidates.sort(key=lambda x: x['score'], reverse=True)
    final_list = candidates[:TOP_K]
    
    for c in final_list:
        print(f"B{c['id']:<4} | {c['res_str']:<7} | {c['tag']:<5} | {c['shift']:.2f} Å  | {c['sasa']:.0f}    | {c['score']:.0f}    | KEEP")
            
    hotspots = [f"B{x['id']}" for x in final_list] + [f"A{x['id']}" for x in final_list]
    constraint_string = "[" + ",".join(hotspots) + "]"
    
    print("-" * 70)
    print(f"Final Selection: {len(final_list)} residues (Top-{TOP_K})")
    print("BINDCRAFT CONSTRAINT STRING:")
    print(constraint_string)

analyze_shifts("mstn_clean.pdb", "gdf11_aligned.pdb", "interface_residues.txt")

Verification RMSD: 2.354 Å over 108 CA pairs

Res   | M-G     | Type  | Shift   | SASA  | Score | Verdict
----------------------------------------------------------------------
B68   | GLY-GLY | GEOM  | 7.05 Å  | 54    | 378    | KEEP
B63   | GLN-GLN | GEOM  | 2.47 Å  | 102    | 253    | KEEP
B99   | PRO-PRO | GEOM  | 2.60 Å  | 87    | 226    | KEEP
B29   | TRP-TRP | GEOM  | 3.25 Å  | 57    | 184    | KEEP
B66   | PRO-PRO | GEOM  | 6.51 Å  | 25    | 160    | KEEP
B69   | SER-SER | GEOM  | 5.06 Å  | 30    | 154    | KEEP
B45   | GLY-GLY | GEOM  | 3.74 Å  | 36    | 134    | KEEP
B98   | ILE-ILE | GEOM  | 2.78 Å  | 46    | 128    | KEEP
B27   | PHE-PHE | GEOM  | 3.15 Å  | 40    | 126    | KEEP
B64   | ALA-ALA | GEOM  | 2.85 Å  | 38    | 108    | KEEP
B79   | MET-MET | GEOM  | 2.24 Å  | 42    | 95    | KEEP
B59   | HIS-HIS | GEOM  | 1.86 Å  | 44    | 81    | KEEP
----------------------------------------------------------------------
Final Selection: 12 residues (Top-12)
BINDCRAFT CONSTRAIN

In [8]:
import py3Dmol
import os

view = py3Dmol.view(width=1000, height=800)
if os.path.exists("mstn_clean.pdb"):
    with open("mstn_clean.pdb") as f: view.addModel(f.read(), "pdb")
    view.setStyle({'model': 0}, {'cartoon': {'color': 'cyan'}})
    view.addLabel("Myostatin", {'position': {'x':0,'y':0,'z':0}, 'backgroundColor':'cyan', 'fontColor':'black'})

if os.path.exists("gdf11_aligned.pdb"):
    with open("gdf11_aligned.pdb") as f: view.addModel(f.read(), "pdb")
    view.setStyle({'model': 1}, {'cartoon': {'color': 'magenta'}})
    view.addLabel("GDF11", {'position': {'x':0,'y':5,'z':0}, 'backgroundColor':'magenta', 'fontColor':'white'})

view.setBackgroundColor('white')
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.